In [ ]:
!pip install pandas
!pip install sklearn
!pip install xgboost

In [ ]:
import pandas as pd

In [ ]:
##"https://docs.google.com/spreadsheets/d/1_PNPgD9gfckqJDMPH7Rcn8wyY5zHrQKE9I1KoLrZbZE/edit?usp=sharing"
#sheet_id = "1_PNPgD9gfckqJDMPH7Rcn8wyY5zHrQKE9I1KoLrZbZE"
#sheet_name = "CriteoSearchData_partner_C0F515F0A2D0A5D9F854008BA76EB537"
#url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"

In [ ]:
#https://docs.google.com/spreadsheets/d/1OKjhawrM8ZAmmpN18yOI0SRhlrhBdl667E30mm9_xb8/edit?usp=sharing
sheet_id = "1OKjhawrM8ZAmmpN18yOI0SRhlrhBdl667E30mm9_xb8"
sheet_name = "CriteoSearchData_partner_C0F515F0A2D0A5D9F854008BA76EB537_new"
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"

In [ ]:
pd.read_csv(url)

In [ ]:
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None

from xgboost.sklearn import XGBRegressor
from sklearn.preprocessing import OneHotEncoder

from itertools import combinations

from sklearn.metrics import mean_squared_error, mean_absolute_error
from random import sample
import matplotlib.pyplot as plt

In [ ]:
def read_CPSD_per_partner_data_group_CSV_file(partner_ID, nrows=None):

    raw_Header_Information = "<Sale> ,<SalesAmountInEuro> ,<time_delay_for_conversion>, <click_timestamp> ,<nb_clicks_1week> ,<product_price> ,<product_age_group> ,<device_type>,<audience_id> ,<product_gender> ,<product_brand> ,<product_category(1-7)> ,<product_country>, <product_id> ,<product_title> ,<partner_id> ,<user_id>"
    product_category_list = []
    for product_category_index in range(7):
        temp_str = "<product_category_" + str(product_category_index + 1) + ">"
        product_category_list.append(temp_str)
    raw_Header_Information = raw_Header_Information.replace(" ", "")
    raw_Header_Information = raw_Header_Information.replace(" ", "")
    raw_Header_Information = raw_Header_Information.replace("<", "")
    raw_Header_Information = raw_Header_Information.replace(">", "")
    Header_Information = raw_Header_Information.split(",")
    header_info = Header_Information[:11] + product_category_list + Header_Information[12:]

    dtypes = {'Sale': 'int64', 'SalesAmountInEuro': 'string', 'time_delay_for_conversion': 'string',
                   'click_timestamp': 'int64', 'nb_clicks_1week': 'string', 'product_price': 'string',
                   'product_age_group': 'string', 'device_type': 'string', 'audience_id': 'string',
                   'product_gender': 'string', 'product_brand': 'string', '<product_category_1>': 'string',
                   '<product_category_2>': 'string', '<product_category_3>': 'string', '<product_category_4>': 'string',
                   '<product_category_5>': 'string', '<product_category_6>': 'string', '<product_category_7>': 'string',
                   'product_country': 'string', 'product_id': 'string', 'product_title': 'string',
                   'partner_id': 'string', 'user_id': 'string'}

    #temp_file_path = "CPSD_per-partner_data_groups"
    temp_file_path = ""
    temp_file_path += "CriteoSearchData_partner_" + partner_ID + "_new.csv"
    #CPSD_per_partner_data_df = pd.read_csv(temp_file_path, sep="\t", nrows=nrows, header=0, names=header_info, dtype=dtypes)
    CPSD_per_partner_data_df = pd.read_csv(url, nrows=nrows, dtype=dtypes)
    #CPSD_per_partner_data_df = pd.read_csv(url, nrows=nrows)
    return CPSD_per_partner_data_df

def make_product_df(partner_ID="C0F515F0A2D0A5D9F854008BA76EB537"):
    CPSD_per_partner_data_df = read_CPSD_per_partner_data_group_CSV_file(partner_ID)
    #CPSD_per_partner_data_df['product_price'] = CPSD_per_partner_data_df['product_price'].fillna(0.0)
    useful_rows_mask = CPSD_per_partner_data_df["product_price"] != "0.0"
    temp_product_df = CPSD_per_partner_data_df[useful_rows_mask]
    print("CPSD_per_partner_data_df.shape: ", CPSD_per_partner_data_df.shape)
    print("temp_product_df.shape: ", temp_product_df.shape)
    #temp_product_df["product_price"] = temp_product_df["product_price"].astype("float")
    temp_product_df["product_price"] = pd.to_numeric(temp_product_df["product_price"], errors='coerce')
    useful_rows_mask = temp_product_df["product_price"].isna()
    temp_product_df = temp_product_df[~useful_rows_mask]
    print("temp_product_df.shape: ", temp_product_df.shape)
    all_raw_columns = list(CPSD_per_partner_data_df)
    print("all_raw_columns: ", all_raw_columns)
    useful_columns = ["product_price", "product_age_group", "product_gender", "product_brand", "product_id"]
    # useful_columns_prefixes = ["<product_category_"]
    additional_useful_columns = []
    for raw_column in all_raw_columns:
        if raw_column.find("<product_category_") != -1:
            if raw_column not in ["<product_category_6", "<product_category_7"]:
              additional_useful_columns.append(raw_column)
    useful_columns += additional_useful_columns
    print("useful_columns: ", useful_columns)
    product_title_df = temp_product_df["product_title"].str.split(" ")
    product_title_expl_df = product_title_df.apply(pd.Series)
    temp_col_rename_dict = {}
    product_title_features = []
    for df_expl_col_name in list(product_title_expl_df):
        temp_new_product_feature_name = "product_title_part_" + str(df_expl_col_name + 1)
        temp_col_rename_dict[df_expl_col_name] = temp_new_product_feature_name
        # product_features.append(temp_new_product_feature_name)
        product_title_features.append(temp_new_product_feature_name)
    product_title_expl_df.rename(columns=temp_col_rename_dict, inplace=True)
    temp_product_df.drop(columns=["product_title"], inplace=True)
    print("about to reindex...")
    product_df = pd.concat([temp_product_df, product_title_expl_df.reindex(product_title_expl_df.index)], axis=1)
    print(list(product_df))
    print("useful_columns: ", useful_columns)
    product_df = product_df[useful_columns]
    return product_df

def make_X_and_y(partner_ID="C0F515F0A2D0A5D9F854008BA76EB537"):
    product_df = make_product_df(partner_ID)
    list_of_column_names = list(product_df)
    list_of_feature_names = list(set(list_of_column_names).difference(set(["product_price"])))
    X_df = product_df[list_of_feature_names]
    y_series = product_df["product_price"]
    X_df = X_df.astype("str")
    #X_array = np.array(X_df)
    X_array = X_df.to_numpy()
    X_array = np.nan_to_num(X_array)
    OH_encoder = OneHotEncoder(sparse=False)
    OH_encoder.fit(X_array)
    OH_encoded_X_dense_matrix = OH_encoder.transform(X_array)
    y_array = np.array(y_series)
    return OH_encoded_X_dense_matrix, y_array


OH_encoded_X, y = make_X_and_y()
test_Regressor = XGBRegressor()
test_Regressor.fit(X=OH_encoded_X, y=y)

In [ ]:
import seaborn as sns

In [ ]:
product_df = make_product_df()

In [ ]:
sns.pairplot(product_df)

In [ ]:
product_df.dtypes

In [ ]:
OH_encoded_X

In [ ]:
# Loosely inspired by sample bootstraping [https://en.wikipedia.org/wiki/Bootstrap_aggregating]


class IndiciesBootstrapper:

    def __init__(self, number_of_bootstraps, min_bootstrap_size=2):
        self.number_of_bootstraps = number_of_bootstraps
        self.min_bootstrap_size = min_bootstrap_size
        self.bootstraps = []
        for i in range(number_of_bootstraps):
            self.bootstraps.append([])

    def bootstraps_diversity_first_indices_list(self, indices_list):
        indices_list_size = len(indices_list)
        bootstrap_size = indices_list_size - 1
        temp_unique_combinations = [comb for comb in combinations(indices_list, bootstrap_size)]
        temp_unique_combinations = np.array(temp_unique_combinations)
        np.random.shuffle(temp_unique_combinations)
        for temp_bootstrap_idx in range(self.number_of_bootstraps):
            self.bootstraps[temp_bootstrap_idx] = temp_unique_combinations[temp_bootstrap_idx].tolist()
        return self.bootstraps

    def classical_independent_bootstraps_indices_list(self, indices_list):
        # temp_buffer = np.array(indices_list)
        # np.random.shuffle(temp_buffer)
        indices_list_size = len(indices_list)
        potential_bootstrap_size = int(indices_list_size/self.number_of_bootstraps)
        bootstrap_size = max(potential_bootstrap_size, self.min_bootstrap_size)
        for temp_bootstrap_idx in range(self.number_of_bootstraps):
            self.bootstraps[temp_bootstrap_idx] = list(np.random.choice(indices_list, bootstrap_size, replace=False))
        return self.bootstraps

    def enhanced_classical_independent_bootstraps_indices_list(self, indices_list):
        # temp_buffer = np.array(indices_list)
        # np.random.shuffle(temp_buffer)
        indices_list_size = len(indices_list)
        no_repetitions_bootstrap_size = int(indices_list_size/self.number_of_bootstraps)
        bootstrap_size = max(no_repetitions_bootstrap_size, self.min_bootstrap_size)
        additional_samples_size = bootstrap_size - no_repetitions_bootstrap_size
        #print("min_bootstrap_size: ", self.min_bootstrap_size)
        #print("bootstrap_size: ", bootstrap_size)
        #print("no_repetitions_bootstrap_size: ", no_repetitions_bootstrap_size)
        #print("additional_samples_size: ", additional_samples_size)
        for temp_bootstrap_idx in range(self.number_of_bootstraps):
            self.bootstraps[temp_bootstrap_idx] = list(np.random.choice(indices_list, no_repetitions_bootstrap_size, replace=False))
            self.bootstraps[temp_bootstrap_idx] += list(np.random.choice(indices_list, additional_samples_size, replace=False))
        return self.bootstraps


#test_indices_list = list(range(3))
test_indices_list = list(range(6))
print("test_indices_list: ", test_indices_list)
number_of_bootstraps = 3
min_bootstrap_size = 4
test_IndiciesBootstrapper = IndiciesBootstrapper(number_of_bootstraps,min_bootstrap_size)
#test_bootstraps = test_IndiciesBootstrapper.bootstraps_diversity_first_indices_list(test_indices_list)
#test_bootstraps = test_IndiciesBootstrapper.classical_independent_bootstraps_indices_list(test_indices_list)
test_bootstraps = test_IndiciesBootstrapper.enhanced_classical_independent_bootstraps_indices_list(test_indices_list)
print()
print("test_bootstraps: ")
print(test_bootstraps)

#Szwabe: What are the differences between the three IndiciesBootstrapper's bootstraping functions/modes? To what extent is the bootstraps independence assumption followed in each of these bootstraping functions/modes? 

150284: IndiciesBootstrapper is used to estimate value functions at so-called indices (states) to bootstrap the estimates in the indices near. Full mode assumes the indices are independent, uses each index' estimate for indices near; Block mode: same schema, but indices are considered correlated; Window mode: indices are correlated, but it uses only indices near. Since independence is connected with blocks/windows used by Block and Window modes (see Note), these two modes follow the bootstraps independence assumption (however, still not fully), whereas in Full mode it is not preserved at all.

**Note:** Larger bootstrapping blocks/windows lead to less independence between estimates.

In [ ]:
class AlternativeSamplesBootstrapper:

    def __init__(self, number_of_bootstraps, min_bootstrap_size):
        self.number_of_bootstraps = number_of_bootstraps
        self.min_bootstrap_size = min_bootstrap_size
        self.bootstraps = {"X":[], "y":[]}
        self.indicies_bootstrapper = IndiciesBootstrapper(self.number_of_bootstraps, self.min_bootstrap_size)

    def make_bootstraps(self, X, y):
        indices_list = list(range(len(X)))
        self.indices_bootstraps = self.indicies_bootstrapper.bootstraps_diversity_first_indices_list(indices_list)
        #print("indices_bootstraps: ", self.indices_bootstraps)
        X_as_array = np.array(X)
        y_as_array = np.array(y)
        self.bootstraps = {"X":[], "y":[]}
        for temp_bootstrap_idx in range(self.number_of_bootstraps):
            #print("self.indices_bootstraps[temp_bootstrap_idx]: ", self.indices_bootstraps[temp_bootstrap_idx)
            #print("np.array(self.indices_bootstraps[temp_bootstrap_idx]): ", np.array(self.indices_bootstraps[temp_bootstrap_idx]))
            #print("X[[7,8,3], :]: ", X[[7,8,3], :])
            self.bootstraps["X"].append(X_as_array[self.indices_bootstraps[temp_bootstrap_idx], :])
            self.bootstraps["y"].append(y_as_array[self.indices_bootstraps[temp_bootstrap_idx]])
            #self.bootstraps[temp_bootstrap_idx] = X[np.array(self.indices_bootstraps[temp_bootstrap_idx]), :]
        return self.bootstraps


test_num_of_samples = 10
number_of_bootstraps = 3
min_bootstrap_size = 2
X, y = make_X_and_y()
test_AlternativeSamplesBootstrapper = AlternativeSamplesBootstrapper(number_of_bootstraps, min_bootstrap_size)
test_AlternativeSamplesBootstrapper.make_bootstraps(X[:test_num_of_samples], y[:test_num_of_samples])

#Szwabe: How is the query strategy implemented?

#What is the purpose of the temp_proba? What does temp_probas indicate? What is the impact of annealing_factor on temp_probas?

150284: temp_proba is a parameter that is used to control the level of exploration during the committee-based learning. Higher temp_proba value leads to more exploratory policy, whereas a lower temp_proba value leads to more greedy policy. Greedy policy makes it more likely to sample from the best performing estimate, whereas exploratory one - from all the estimates in the committee. So that temp_proba to some extent is in charge of exploration-exploitation trade-off during commitee-based learning. The annealing_factor parameter is used to reduce the value of temp_proba during the learning. It shifts the learning from exploration to exploitation over time.

**Note:** Committee-based learning involves sampling from the different estimates to select an action.

In [ ]:
class AlternativeCommitteeRegressor:

    def __init__(self, variance_unaware_regressor_class, number_of_regressors_in_commitee):
        self.number_of_regressors_in_committee = number_of_regressors_in_commitee
        self.min_bootstrap_size = 2 # temporary
        self.committee = []
        for regressor_idx in range(self.number_of_regressors_in_committee):
            #self.committee.append(variance_unaware_regressor_class())
            self.committee.append(variance_unaware_regressor_class(verbosity = 0))
        self.X = []
        self.y = []
        self.samples_bootstraper = AlternativeSamplesBootstrapper(self.number_of_regressors_in_committee, self.min_bootstrap_size)

    def _distribute_X_and_y(self):
        pass

    def teach(self, X, y):
        self.X.append(X)
        self.y.append(y)

    def _make_bootstraps(self):
        self.bootstraps = self.samples_bootstraper.make_bootstraps(self.X, self.y)

    def _committee_train(self):
        for regressor_idx in range(self.number_of_regressors_in_committee):
            self.committee[regressor_idx].fit(X=self.bootstraps["X"][regressor_idx], y=self.bootstraps["y"][regressor_idx])

    def predict(self, X_query):
        self.committee_predictions = []
        for regressor_idx in range(self.number_of_regressors_in_committee):
            self.committee_predictions.append(self.committee[regressor_idx].predict(X_query))

    def _compute_means_and_STDs_for_query(self, X_query):
        self._make_bootstraps()
        self._committee_train()
        self.predict(X_query)

        self.committee_predictions_array = np.array(self.committee_predictions)

        self._committee_means = np.mean(self.committee_predictions_array, axis=0)
        # self.committee_differences = self.committee_predictions_array - self.committee_means
        # self.committee_differences_squared = self.committee_differences**2
        # self.committee_differences_squared_sums = np.sum(self.committee_differences_squared, axis=0)
        # self.committee_differences_squared_means = self.committee_differences_squared_sums/self.committee_differences.shape[0]
        # self.committee_differences_squared_means_roots = np.sqrt(self.committee_differences_squared_means)
        # Szwabe: What would the 5 lines above do if they were not commented?
        '''
        150284: 
        It would estimate variance of the estimates, so that
        it would reduce less certain estimates' weights.
        This could improve the stability.
        '''
        self._committee_STDs = np.std(self.committee_predictions_array, axis=0)
        #print(self.committee_STDs == self.committee_differences_squared_means_roots)
        #return self.committee_means, self.committee_STDs

    def query(self, X_query):
        self._compute_means_and_STDs_for_query(X_query)
        self.query_result = np.argmax(self._committee_STDs)
        return self.query_result


X, y = make_X_and_y()
shuffled_indices = np.arange(y.shape[0])
np.random.shuffle(shuffled_indices)
y = y[shuffled_indices]
X = X[shuffled_indices]

number_of_regressors_in_commitee = 2
committee = AlternativeCommitteeRegressor(XGBRegressor, number_of_regressors_in_commitee)
number_of_initial_samples = 2 * number_of_regressors_in_commitee #temporary
initial_X_samples = X[:number_of_initial_samples]
initial_y_samples = y[:number_of_initial_samples]
X_query = X[number_of_initial_samples:]
y_query = y[number_of_initial_samples:]
for temp_initial_sample_idx in range(number_of_initial_samples):
  X_train_sample = initial_X_samples[temp_initial_sample_idx]
  y_train_sample = initial_y_samples[temp_initial_sample_idx]
  committee.teach(X_train_sample, y_train_sample)
query_result = committee.query(X_query)
MSE_scores_of_AlternativeCommitteeRegressor = []
MAE_scores_of_AlternativeCommitteeRegressor = []
temp_probas = []
annealing_factor = 0.04
for i in range(200):
  query_result = committee.query(X_query)
  temp_prob = np.random.rand()
  temp_prob = np.log(temp_prob)
  temp_prob = temp_prob / (annealing_factor * i + 1)
  temp_prob = int(round(temp_prob)) % 2
  temp_probas.append(temp_prob)
  print("temp_prob: ", temp_prob)
  if temp_prob > 0.5:
      query_result = sample(list(range(X_query.shape[0])), 1)[0]
      print("this time it's random")
  print("query_result: ", query_result)
  temp_committee_predictions = committee._committee_means
  temp_score = mean_squared_error(y_query, temp_committee_predictions)
  MSE_scores_of_AlternativeCommitteeRegressor.append(temp_score)
  temp_score = mean_absolute_error(y_query, temp_committee_predictions)
  MAE_scores_of_AlternativeCommitteeRegressor.append(temp_score)
  another_teaching_sample_X = X_query[query_result]
  another_teaching_sample_y = y_query[query_result]
  committee.teach(another_teaching_sample_X, another_teaching_sample_y)
  X_query = X_query[np.arange(len(X_query))!=query_result]
  y_query = y_query[np.arange(len(y_query))!=query_result]

committee = AlternativeCommitteeRegressor(XGBRegressor, number_of_regressors_in_commitee)
number_of_initial_samples = 2 * number_of_regressors_in_commitee #temporary
initial_X_samples = X[:number_of_initial_samples]
initial_y_samples = y[:number_of_initial_samples]
X_query = X[number_of_initial_samples:]
y_query = y[number_of_initial_samples:]
for temp_initial_sample_idx in range(number_of_initial_samples):
    X_train_sample = initial_X_samples[temp_initial_sample_idx]
    y_train_sample = initial_y_samples[temp_initial_sample_idx]
    committee.teach(X_train_sample, y_train_sample)
query_result = committee.query(X_query)

MSE_scores_of_random_baseline = []
MAE_scores_of_random_baseline = []
for i in range(200):
    query_result = committee.query(X_query)
    query_result = sample(list(range(X_query.shape[0])), 1)[0]
    print("query_result: ", query_result)
    temp_committee_predictions = committee._committee_means
    temp_score = mean_squared_error(y_query, temp_committee_predictions)
    MSE_scores_of_random_baseline.append(temp_score)
    temp_score = mean_absolute_error(y_query, temp_committee_predictions)
    MAE_scores_of_random_baseline.append(temp_score)
    another_teaching_sample_X = X_query[query_result]
    another_teaching_sample_y = y_query[query_result]
    committee.teach(another_teaching_sample_X, another_teaching_sample_y)
    X_query = X_query[np.arange(len(X_query))!=query_result]
    y_query = y_query[np.arange(len(y_query))!=query_result]

In [ ]:
plt.plot(MSE_scores_of_AlternativeCommitteeRegressor, label="AlternativeCommitteeRegressor")
plt.plot(MSE_scores_of_random_baseline, label="Random")
plt.legend()
plt.show()

In [ ]:
plt.clf()
plt.plot(MAE_scores_of_AlternativeCommitteeRegressor, label="AlternativeCommitteeRegressor")
plt.plot(MAE_scores_of_random_baseline, label="Random")
plt.legend()
plt.show()

In [ ]:
    plt.clf()
    plt.plot(temp_probas, label="temp_probas")
    plt.legend()
    plt.show()

In [ ]:
def test_comparing_with_baseline(X, y, annealing_factor, number_of_regressors_in_commitee, number_of_cycles):
  shuffled_indices = np.arange(y.shape[0])
  np.random.shuffle(shuffled_indices)
  y = y[shuffled_indices]
  X = X[shuffled_indices]

  committee = AlternativeCommitteeRegressor(XGBRegressor, number_of_regressors_in_commitee)
  number_of_initial_samples = 2 * number_of_regressors_in_commitee #temporary
  initial_X_samples = X[:number_of_initial_samples]
  initial_y_samples = y[:number_of_initial_samples]
  X_query = X[number_of_initial_samples:]
  y_query = y[number_of_initial_samples:]
  for temp_initial_sample_idx in range(number_of_initial_samples):
    X_train_sample = initial_X_samples[temp_initial_sample_idx]
    y_train_sample = initial_y_samples[temp_initial_sample_idx]
    committee.teach(X_train_sample, y_train_sample)
  query_result = committee.query(X_query)
  MSE_scores_of_AlternativeCommitteeRegressor = []
  MAE_scores_of_AlternativeCommitteeRegressor = []
  temp_probas = []
  for i in range(number_of_cycles):
    query_result = committee.query(X_query)
    temp_prob = np.random.rand()
    temp_prob = np.log(temp_prob)
    temp_prob = temp_prob / (annealing_factor * i + 1)
    temp_prob = int(round(temp_prob)) % 2
    temp_probas.append(temp_prob)
    print("temp_prob: ", temp_prob)
    if temp_prob > 0.5:
        query_result = sample(list(range(X_query.shape[0])), 1)[0]
        print("this time it's random")
    print("query_result: ", query_result)
    temp_committee_predictions = committee._committee_means
    temp_score = mean_squared_error(y_query, temp_committee_predictions)
    MSE_scores_of_AlternativeCommitteeRegressor.append(temp_score)
    temp_score = mean_absolute_error(y_query, temp_committee_predictions)
    MAE_scores_of_AlternativeCommitteeRegressor.append(temp_score)
    another_teaching_sample_X = X_query[query_result]
    another_teaching_sample_y = y_query[query_result]
    committee.teach(another_teaching_sample_X, another_teaching_sample_y)
    X_query = X_query[np.arange(len(X_query))!=query_result]
    y_query = y_query[np.arange(len(y_query))!=query_result]
  committee = AlternativeCommitteeRegressor(XGBRegressor, number_of_regressors_in_commitee)
  number_of_initial_samples = 2 * number_of_regressors_in_commitee #temporary
  initial_X_samples = X[:number_of_initial_samples]
  initial_y_samples = y[:number_of_initial_samples]
  X_query = X[number_of_initial_samples:]
  y_query = y[number_of_initial_samples:]
  for temp_initial_sample_idx in range(number_of_initial_samples):
      X_train_sample = initial_X_samples[temp_initial_sample_idx]
      y_train_sample = initial_y_samples[temp_initial_sample_idx]
      committee.teach(X_train_sample, y_train_sample)
  query_result = committee.query(X_query)
  MSE_scores_of_random_baseline = []
  MAE_scores_of_random_baseline = []
  for i in range(number_of_cycles):
      query_result = committee.query(X_query)
      query_result = sample(list(range(X_query.shape[0])), 1)[0]
      print("query_result: ", query_result)
      temp_committee_predictions = committee._committee_means
      temp_score = mean_squared_error(y_query, temp_committee_predictions)
      MSE_scores_of_random_baseline.append(temp_score)
      temp_score = mean_absolute_error(y_query, temp_committee_predictions)
      MAE_scores_of_random_baseline.append(temp_score)
      another_teaching_sample_X = X_query[query_result]
      another_teaching_sample_y = y_query[query_result]
      committee.teach(another_teaching_sample_X, another_teaching_sample_y)
      X_query = X_query[np.arange(len(X_query))!=query_result]
      y_query = y_query[np.arange(len(y_query))!=query_result]
  return MSE_scores_of_AlternativeCommitteeRegressor, MSE_scores_of_random_baseline, MAE_scores_of_AlternativeCommitteeRegressor, MAE_scores_of_random_baseline

In [ ]:
def multiple_tests_comparing_with_baseline(X, y, number_of_tests, annealing_factor, number_of_regressors_in_commitee, number_of_cycles):
  means_and_stds = {"MAE": {}, "MSE": {}}
  MSE_scores_of_AlternativeCommitteeRegressor_list = []
  MSE_scores_of_random_baseline_list = []
  MAE_scores_of_AlternativeCommitteeRegressor_list = []
  MAE_scores_of_random_baseline_list = []
  for test_number in range(number_of_tests):
    MSE_scores_of_AlternativeCommitteeRegressor, MSE_scores_of_random_baseline, MAE_scores_of_AlternativeCommitteeRegressor, MAE_scores_of_random_baseline = test_comparing_with_baseline(X, y, annealing_factor, number_of_regressors_in_commitee, number_of_cycles)
    MSE_scores_of_AlternativeCommitteeRegressor_list.append(MSE_scores_of_AlternativeCommitteeRegressor)
    MSE_scores_of_random_baseline_list.append(MSE_scores_of_random_baseline)
    MAE_scores_of_AlternativeCommitteeRegressor_list.append(MAE_scores_of_AlternativeCommitteeRegressor)
    MAE_scores_of_random_baseline_list.append(MAE_scores_of_random_baseline)
  temp_mean = np.mean(np.array(MSE_scores_of_AlternativeCommitteeRegressor_list), axis=0)
  temp_std = np.std(np.array(MSE_scores_of_AlternativeCommitteeRegressor_list), axis=0)
  means_and_stds["MSE"]["AlternativeCommitteeRegressor"] = {"mean": temp_mean, "std": temp_std}
  temp_mean = np.mean(np.array(MAE_scores_of_AlternativeCommitteeRegressor_list), axis=0)
  temp_std = np.std(np.array(MAE_scores_of_AlternativeCommitteeRegressor_list), axis=0)
  means_and_stds["MAE"]["AlternativeCommitteeRegressor"] = {"mean": temp_mean, "std": temp_std}
  temp_mean = np.mean(np.array(MSE_scores_of_random_baseline_list), axis=0)
  temp_std = np.std(np.array(MSE_scores_of_random_baseline_list), axis=0)
  means_and_stds["MSE"]["random_baseline"] = {"mean": temp_mean, "std": temp_std}
  temp_mean = np.mean(np.array(MAE_scores_of_random_baseline_list), axis=0)
  temp_std = np.std(np.array(MAE_scores_of_random_baseline_list), axis=0)
  means_and_stds["MAE"]["random_baseline"] = {"mean": temp_mean, "std": temp_std}
  MSE_scores_of_random_baseline_mean = np.mean(np.array(MSE_scores_of_random_baseline_list), axis=0)
  MSE_scores_of_random_baseline_std = np.std(np.array(MSE_scores_of_random_baseline_list), axis=0)
  return means_and_stds

In [ ]:
def plot_means_and_stds(means_and_stds, temp_measure):
  plt.clf
  plt.rcParams["figure.figsize"] = (20,14)
  temp_color_letters = ["b", "y"]
  #temp_measure = "MAE"
  temp_measure_data = means_and_stds[temp_measure]
  temp_algorithms_labes = list(temp_measure_data)
  for temp_algorithm_label_idx, temp_algorithm_label in enumerate(temp_algorithms_labes):
    temp_color_letter = temp_color_letters[temp_algorithm_label_idx]
    temp_mean = means_and_stds[temp_measure][temp_algorithm_label]["mean"]
    temp_std = means_and_stds[temp_measure][temp_algorithm_label]["std"]
    x = np.arange(len(temp_mean))
    plt.plot(x, temp_mean, temp_color_letter+"-", label=temp_algorithm_label)
    plt.fill_between(x, temp_mean - temp_std, temp_mean + temp_std, color=temp_color_letter, alpha=0.2)
  plt.legend()

#Szwabe: Set an appropriate number of AR cycles and test repetitions (below) in order to compare practical value of the sampling strategies in the most reliable way that is still feasible considering the amount of available time (of the classwork). Compare and comment on the practical value of the AR solution. What is the true practical value of the temp_proba for different values of annealing_factor? Can you make the active regressor significantly (and reliably) more effective than random sampling by using some specific combination of bootstraping function/mode and annealing_factor value? Note all the observations you make.

150284: We need bigger number_of_tests using small annealing factor (such as 0.03, e.g.). However, we could also lower the number_of_tests to around 20 using annealing_factor of 0.1 at the time. It gives approximately the same results, since higher annealing_factor makes the exploitation faster. It is also possible to lower the number_of_cyces to 100 to achieve approximately the same results. It indeed is possible to make the active regressor significantly (and reliably) more effective than random sampling by using some specific combination of bootstraping function/mode and annealing_factor value. We could start with temp_proba of 1, making it firstly completely exploitational. And then we could periodically decrease annealing_factor value, so that it would reduce temp_proba and lead to more of the exploration. The number_of_cycles increases the running time a lot, so that I didn't manage to test it for the  value bigger than 200.

In [ ]:
number_of_tests = 25
# number_of_tests = 4 # Warning: ridiculously low number
annealing_factor = 0.06
number_of_regressors_in_commitee = 2
number_of_cycles = 100

means_and_stds = multiple_tests_comparing_with_baseline(X, y, number_of_tests, annealing_factor, number_of_regressors_in_commitee, number_of_cycles)

In [ ]:
plot_means_and_stds(means_and_stds, "MSE")

In [ ]:
plot_means_and_stds(means_and_stds, "MAE")

In [ ]:
number_of_tests = 25
# number_of_tests = 4 # Warning: ridiculously low number
annealing_factor = 0.08
number_of_regressors_in_commitee = 2
number_of_cycles = 100

means_and_stds = multiple_tests_comparing_with_baseline(X, y, number_of_tests, annealing_factor, number_of_regressors_in_commitee, number_of_cycles)

In [ ]:
plot_means_and_stds(means_and_stds, "MSE")

In [ ]:
plot_means_and_stds(means_and_stds, "MAE")

In [ ]:
number_of_tests = 25
# number_of_tests = 4 # Warning: ridiculously low number
annealing_factor = 0.1
number_of_regressors_in_commitee = 2
number_of_cycles = 100

means_and_stds = multiple_tests_comparing_with_baseline(X, y, number_of_tests, annealing_factor, number_of_regressors_in_commitee, number_of_cycles)

In [ ]:
plot_means_and_stds(means_and_stds, "MSE")

In [ ]:
plot_means_and_stds(means_and_stds, "MAE")

In [ ]:
!pip freeze

absl-py==1.0.0
alabaster==0.7.12
albumentations==0.1.12
altair==4.2.0
appdirs==1.4.4
argon2-cffi==21.3.0
argon2-cffi-bindings==21.2.0
arviz==0.12.0
astor==0.8.1
astropy==4.3.1
astunparse==1.6.3
atari-py==0.2.9
atomicwrites==1.4.0
attrs==21.4.0
audioread==2.1.9
autograd==1.4
Babel==2.9.1
backcall==0.2.0
beautifulsoup4==4.6.3
bleach==5.0.0
blis==0.4.1
bokeh==2.3.3
Bottleneck==1.3.4
branca==0.4.2
bs4==0.0.1
CacheControl==0.12.10
cached-property==1.5.2
cachetools==4.2.4
catalogue==1.0.0
certifi==2021.10.8
cffi==1.15.0
cftime==1.6.0
chardet==3.0.4
charset-normalizer==2.0.12
click==7.1.2
cloudpickle==1.3.0
cmake==3.12.0
cmdstanpy==0.9.5
colorcet==3.0.0
colorlover==0.3.0
community==1.0.0b1
contextlib2==0.5.5
convertdate==2.4.0
coverage==3.7.1
coveralls==0.5
crcmod==1.7
cufflinks==0.17.3
cvxopt==1.2.7
cvxpy==1.0.31
cycler==0.11.0
cymem==2.0.6
Cython==0.29.28
daft==0.0.4
dask==2.12.0
datascience==0.10.6
debugpy==1.0.0
decorator==4.4.2
defusedxml==0.7.1
descartes==1.1.0
dill==0.3.4
distributed==1.25.3
dlib @ file:///dlib-19.18.0-cp37-cp37m-linux_x86_64.whl
dm-tree==0.1.7
docopt==0.6.2
docutils==0.17.1
dopamine-rl==1.0.5
earthengine-api==0.1.306
easydict==1.9
ecos==2.0.10
editdistance==0.5.3
en-core-web-sm @ https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.2.5/en_core_web_sm-2.2.5.tar.gz
entrypoints==0.4
ephem==4.1.3
et-xmlfile==1.1.0
fa2==0.3.5
fastai==1.0.61
fastdtw==0.3.4
fastjsonschema==2.15.3
fastprogress==1.0.2
fastrlock==0.8
fbprophet==0.7.1
feather-format==0.4.1
filelock==3.6.0
firebase-admin==4.4.0
fix-yahoo-finance==0.0.22
Flask==1.1.4
flatbuffers==2.0
folium==0.8.3
future==0.16.0
gast==0.5.3
GDAL==2.2.2
gdown==4.4.0
gensim==3.6.0
geographiclib==1.52
geopy==1.17.0
gin-config==0.5.0
glob2==0.7
google==2.0.3
google-api-core==1.31.5
google-api-python-client==1.12.11
google-auth==1.35.0
google-auth-httplib2==0.0.4
google-auth-oauthlib==0.4.6
google-cloud-bigquery==1.21.0
google-cloud-bigquery-storage==1.1.1
google-cloud-core==1.0.3
google-cloud-datastore==1.8.0
google-cloud-firestore==1.7.0
google-cloud-language==1.2.0
google-cloud-storage==1.18.1
google-cloud-translate==1.5.0
google-colab @ file:///colabtools/dist/google-colab-1.0.0.tar.gz
google-pasta==0.2.0
google-resumable-media==0.4.1
googleapis-common-protos==1.56.0
googledrivedownloader==0.4
graphviz==0.10.1
greenlet==1.1.2
grpcio==1.44.0
gspread==3.4.2
gspread-dataframe==3.0.8
gym==0.17.3
h5py==3.1.0
HeapDict==1.0.1
hijri-converter==2.2.3
holidays==0.10.5.2
holoviews==1.14.8
html5lib==1.0.1
httpimport==0.5.18
httplib2==0.17.4
httplib2shim==0.0.3
humanize==0.5.1
hyperopt==0.1.2
ideep4py==2.0.0.post3
idna==2.10
imageio==2.4.1
imagesize==1.3.0
imbalanced-learn==0.8.1
imblearn==0.0
imgaug==0.2.9
importlib-metadata==4.11.3
importlib-resources==5.7.0
imutils==0.5.4
inflect==2.1.0
iniconfig==1.1.1
intel-openmp==2022.0.2
intervaltree==2.1.0
ipykernel==4.10.1
ipython==5.5.0
ipython-genutils==0.2.0
ipython-sql==0.3.9
ipywidgets==7.7.0
itsdangerous==1.1.0
jax==0.3.4
jaxlib @ https://storage.googleapis.com/jax-releases/cuda11/jaxlib-0.3.2+cuda11.cudnn805-cp37-none-manylinux2010_x86_64.whl
jedi==0.18.1
jieba==0.42.1
Jinja2==2.11.3
joblib==1.1.0
jpeg4py==0.1.4
jsonschema==4.3.3
jupyter==1.0.0
jupyter-client==5.3.5
jupyter-console==5.2.0
jupyter-core==4.9.2
jupyterlab-pygments==0.2.2
jupyterlab-widgets==1.1.0
kaggle==1.5.12
kapre==0.3.7
keras==2.8.0
Keras-Preprocessing==1.1.2
keras-vis==0.4.1
kiwisolver==1.4.2
korean-lunar-calendar==0.2.1
libclang==13.0.0
librosa==0.8.1
lightgbm==2.2.3
llvmlite==0.34.0
lmdb==0.99
LunarCalendar==0.0.9
lxml==4.2.6
Markdown==3.3.6
MarkupSafe==2.0.1
matplotlib==3.2.2
matplotlib-inline==0.1.3
matplotlib-venn==0.11.7
missingno==0.5.1
mistune==0.8.4
mizani==0.6.0
mkl==2019.0
mlxtend==0.14.0
more-itertools==8.12.0
moviepy==0.2.3.5
mpmath==1.2.1
msgpack==1.0.3
multiprocess==0.70.12.2
multitasking==0.0.10
murmurhash==1.0.6
music21==5.5.0
natsort==5.5.0
nbclient==0.6.0
nbconvert==5.6.1
nbformat==5.3.0
nest-asyncio==1.5.5
netCDF4==1.5.8
networkx==2.6.3
nibabel==3.0.2
nltk==3.2.5
notebook==5.3.1
numba==0.51.2
numexpr==2.8.1
numpy==1.21.6
nvidia-ml-py3==7.352.0
oauth2client==4.1.3
oauthlib==3.2.0
okgrade==0.4.3
opencv-contrib-python==4.1.2.30
opencv-python==4.1.2.30
openpyxl==3.0.9
opt-einsum==3.3.0
osqp==0.6.2.post0
packaging==21.3
palettable==3.3.0
pandas==1.3.5
pandas-datareader==0.9.0
pandas-gbq==0.13.3
pandas-profiling==1.4.1
pandocfilters==1.5.0
panel==0.12.1
param==1.12.1
parso==0.8.3
pathlib==1.0.1
patsy==0.5.2
pep517==0.12.0
pexpect==4.8.0
pickleshare==0.7.5
Pillow==7.1.2
pip-tools==6.2.0
plac==1.1.3
plotly==5.5.0
plotnine==0.6.0
pluggy==0.7.1
pooch==1.6.0
portpicker==1.3.9
prefetch-generator==1.0.1
preshed==3.0.6
prettytable==3.2.0
progressbar2==3.38.0
prometheus-client==0.14.1
promise==2.3
prompt-toolkit==1.0.18
protobuf==3.17.3
psutil==5.4.8
psycopg2==2.7.6.1
ptyprocess==0.7.0
py==1.11.0
pyarrow==6.0.1
pyasn1==0.4.8
pyasn1-modules==0.2.8
pycocotools==2.0.4
pycparser==2.21
pyct==0.4.8
pydata-google-auth==1.4.0
pydot==1.3.0
pydot-ng==2.0.0
pydotplus==2.0.2
PyDrive==1.3.1
pyemd==0.5.1
pyerfa==2.0.0.1
pyglet==1.5.0
Pygments==2.6.1
pygobject==3.26.1
pymc3==3.11.4
PyMeeus==0.5.11
pymongo==4.1.1
pymystem3==0.2.0
PyOpenGL==3.1.6
pyparsing==3.0.8
pyrsistent==0.18.1
pysndfile==1.3.8
PySocks==1.7.1
pystan==2.19.1.1
pytest==3.6.4
python-apt==0.0.0
python-chess==0.23.11
python-dateutil==2.8.2
python-louvain==0.16
python-slugify==6.1.1
python-utils==3.1.0
pytz==2022.1
pyviz-comms==2.2.0
PyWavelets==1.3.0
PyYAML==3.13
pyzmq==22.3.0
qdldl==0.1.5.post2
qtconsole==5.3.0
QtPy==2.0.1
regex==2019.12.20
requests==2.23.0
requests-oauthlib==1.3.1
resampy==0.2.2
rpy2==3.4.5
rsa==4.8
scikit-image==0.18.3
scikit-learn==1.0.2
scipy==1.4.1
screen-resolution-extra==0.0.0
scs==3.2.0
seaborn==0.11.2
semver==2.13.0
Send2Trash==1.8.0
setuptools-git==1.2
Shapely==1.8.1.post1
simplegeneric==0.8.1
six==1.15.0
sklearn==0.0
sklearn-pandas==1.8.0
smart-open==5.2.1
snowballstemmer==2.2.0
sortedcontainers==2.4.0
SoundFile==0.10.3.post1
soupsieve==2.3.2.post1
spacy==2.2.4
Sphinx==1.8.6
sphinxcontrib-serializinghtml==1.1.5
sphinxcontrib-websupport==1.2.4
SQLAlchemy==1.4.35
sqlparse==0.4.2
srsly==1.0.5
statsmodels==0.10.2
sympy==1.7.1
tables==3.7.0
tabulate==0.8.9
tblib==1.7.0
tenacity==8.0.1
tensorboard==2.8.0
tensorboard-data-server==0.6.1
tensorboard-plugin-wit==1.8.1
tensorflow @ file:///tensorflow-2.8.0-cp37-cp37m-linux_x86_64.whl
tensorflow-datasets==4.0.1
tensorflow-estimator==2.8.0
tensorflow-gcs-config==2.8.0
tensorflow-hub==0.12.0
tensorflow-io-gcs-filesystem==0.24.0
tensorflow-metadata==1.7.0
tensorflow-probability==0.16.0
termcolor==1.1.0
terminado==0.13.3
testpath==0.6.0
text-unidecode==1.3
textblob==0.15.3
Theano-PyMC==1.1.2
thinc==7.4.0
threadpoolctl==3.1.0
tifffile==2021.11.2
tinycss2==1.1.1
tomli==2.0.1
toolz==0.11.2
torch @ https://download.pytorch.org/whl/cu111/torch-1.10.0%2Bcu111-cp37-cp37m-linux_x86_64.whl
torchaudio @ https://download.pytorch.org/whl/cu111/torchaudio-0.10.0%2Bcu111-cp37-cp37m-linux_x86_64.whl
torchsummary==1.5.1
torchtext==0.11.0
torchvision @ https://download.pytorch.org/whl/cu111/torchvision-0.11.1%2Bcu111-cp37-cp37m-linux_x86_64.whl
tornado==5.1.1
tqdm==4.64.0
traitlets==5.1.1
tweepy==3.10.0
typeguard==2.7.1
typing-extensions==4.1.1
tzlocal==1.5.1
uritemplate==3.0.1
urllib3==1.24.3
vega-datasets==0.9.0
wasabi==0.9.1
wcwidth==0.2.5
webencodings==0.5.1
Werkzeug==1.0.1
widgetsnbextension==3.6.0
wordcloud==1.5.0
wrapt==1.14.0
xarray==0.18.2
xgboost==0.90
xkit==0.0.0
xlrd==1.1.0
xlwt==1.3.0
yellowbrick==1.4
zict==2.1.0
zipp==3.8.0